In [4]:
#RoBERTa is BERT variation but following changes
''' -Use dynamic mask, instead of static mask
     -Remove the NSP task and train using the MLM task
     -Train with latge batch size
     -use byte-level BPE(BBPE) as a tokenizer
     '''

' -Use dynamic mask, instead of static mask\n     -Remove the NSP task and train using the MLM task\n     -Train with latge batch size\n     -use byte-level BPE(BBPE) as a tokenizer\n     '

In [5]:
#SOP basically a classification task, classify pair of words belongs to positive or negative class(Order of word not swapped)
 #or negative class(order of word swapped)

In [6]:
from transformers import RobertaConfig, RobertaModel, RobertaModel, RobertaTokenizer
import pandas as pd
import numpy as np
import nltk
import torch

In [122]:
roberta=RobertaModel.from_pretrained('roberta-base',output_hidden_states=True)
roberta.config

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [8]:
tokenizer=RobertaTokenizer.from_pretrained('roberta-base')

In [9]:
path=r"C:\Users\uttam\OneDrive\Desktop\NLP Project\NLP-Project\NLP-Dataset.csv"
df=pd.read_csv(path,sep=None,engine='python')
df

,sentence,label
0,My name is uttam asodariya,0
1,I am ambitious student,1
2,I like travelling,0
3,I cannot here you,1


In [10]:
train_data=df['sentence'].loc[0:2]
test_data=df['sentence'].loc[3:]
train_data

0    My name is uttam asodariya
1        I am ambitious student
2             I like travelling
Name: sentence, dtype: object

In [160]:
def Word_embedding(dataframe):
    for i in dataframe:
        input = torch.tensor(tokenizer.encode(i, add_special_tokens=True)).unsqueeze(0)
        
        with torch.no_grad():
            output = roberta(input)
            last_hidden_states=output[2]
            
            sum_of_encoders=torch.stack(last_hidden_states).sum(0)
            sum_of_encoders=sum_of_encoders/12
            print(sum_of_encoders)
        

In [161]:
Word_embedding(train_data)

tensor([[[ 0.0027,  0.0491, -0.0048,  ...,  0.0226,  0.0048, -0.0221],
         [-0.0660,  0.4460,  0.1020,  ...,  0.1256, -0.2572, -0.1148],
         [-0.1083,  0.5719,  0.0382,  ..., -0.1385, -0.2744, -0.1559],
         ...,
         [ 0.0997, -0.0749,  0.1950,  ..., -1.2354, -0.3278, -0.2891],
         [ 0.0701, -0.5414,  0.1331,  ..., -0.9609, -0.4482, -0.7704],
         [-0.0280,  0.0686,  0.0474,  ...,  0.2652,  0.0256, -0.1421]]])
tensor([[[ 2.9430e-03,  5.1475e-02, -2.2139e-04,  ...,  5.4950e-02,
           4.9662e-03, -1.6725e-02],
         [ 4.3071e-02,  2.5084e-01,  2.9740e-01,  ...,  4.8996e-01,
          -7.6162e-02,  1.8444e-02],
         [ 1.6980e-01,  5.4599e-01,  9.3926e-02,  ...,  4.9652e-02,
          -5.9217e-02,  1.4925e-01],
         [ 8.2230e-02, -5.8170e-01, -1.0042e-02,  ...,  5.0498e-01,
           2.4779e-01,  1.5696e-01],
         [ 2.2071e-01, -5.3641e-01, -3.4504e-01,  ..., -3.3112e-02,
          -2.8576e-01, -1.4333e-01],
         [ 1.3871e-02, -4.7754e-0